In [1]:
!pip install TextX

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 5.1 MB/s eta 0:00:00


In [10]:
#To create a metamodel from a file
from textx import metamodel_from_file
metamodel = metamodel_from_file('EBNF.tx')

#Create a syntax tree using the  metamodel and the program in a file
syntaxTree = metamodel.model_from_file('example2.tx')
#print(metamodel)
syntaxTree




<textx:EBNF.ClassDiagram instance at 0x7d01071dab10>

In [11]:
#To create the metamodel and model diagrams
#filename is a filename with the grammar

#For the metamodel
!textx generate EBNF.tx --target dot




Generating dot target from models:
/content/EBNF.tx
-> /content/EBNF.dot
     To convert to png run "dot -Tpng -O EBNF.dot"


In [12]:
import pydot

(graph,) = pydot.graph_from_dot_file('EBNF.dot')
graph.write_png('EBNF.png')

#For the model, you'll need a file with the source code and another with the grammar


!textx generate example2.tx --grammar EBNF.tx --target dot
(graph,) = pydot.graph_from_dot_file('example2.dot')
graph.write_png('finalExample2.png')



Generating dot target from models:
/content/example2.tx
-> /content/example2.dot
     To convert to png run "dot -Tpng -O example2.dot"


In [14]:
!pip install plantuml

In [15]:
!apt-get install plantuml -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  binfmt-support default-jdk default-jdk-headless default-jre
  default-jre-headless ditaa fastjar fonts-dejavu-core fonts-dejavu-extra
  icc-profiles-free jarwrapper java-wrappers junit4 libapache-pom-java
  libatk-wrapper-java libatk-wrapper-java-jni libavalon-framework-java
  libbatik-java libcommons-cli-java libcommons-io-java libcommons-logging-java
  libcommons-parent-java libfontbox2-java libfop-java libhamcrest-java
  libjaxp1.3-java libjericho-html-java libjlatexmath-java libqdox-java
  libsaxon-java libxalan2-java libxerces2-java libxml-commons-external-java
  libxml-commons-resolver1.1-java libxmlgraphics-commons-java libxt-dev
  libxtst6 libxxf86dga1 openjdk-11-jdk openjdk-11-jre x11-utils
Suggested packages:
  libavalon-framework-java-doc librhino-java libcommons-io-java-doc
  libcommons-logging-java-doc libexcalibur-logkit-j

In [13]:
def to_plantuml(model):
    lines = ["@startuml"]

    for obj in model.objects:
        if obj.objectType.__class__.__name__ == "Class":
            class_obj = obj.objectType

            # nombre de la clase
            name = class_obj.name.upName

            attributes = "\n".join(
                f"    {attr.visibility.visibility}{attr.name.lowName}: {getattr(attr.type, 'datatype', '?')}"
                for attr in getattr(class_obj, "attributes", [])
            )

            methods = "\n".join(f"    {m.visibility.visibility}{m.name.lowName}("+ ", ".join(f"{p.name.lowName}: {p.type.datatype}"
              for p in getattr(m.params, "params", []))+ f"): {m.returnType.retorno.datatype}"
              if m.params and hasattr(m.params, "params")
              else f"    {m.visibility.visibility}{m.name.lowName}(): {m.returnType.retorno.datatype}"
              for m in getattr(class_obj, "methods", [])
            )


              # armar clase completa
            lines.append(f"class {name} {{\n{attributes}\n{methods}\n}}")





    for obj in model.relation:
       rel_type = obj.type.relations.strip("'")
       arrow = {
            "--->": "-->",
            "---": "--",
            #"---<>": "*--",
            #"---<*>": "o--"
        }.get(rel_type, "--")

       start = obj.start.upName.strip()
       end = obj.end.upName.strip()
       startMult = obj.startMultiplicity.multiplicities.strip()
       endMult = obj.endMultiplicity.multiplicities.strip()

       relation_line = f'{start} "{startMult}" {arrow} "{endMult}" {end}'
       lines.append(relation_line)


    lines.append("@enduml")
    return "\n".join(lines)

plantuml_code = to_plantuml(syntaxTree)
with open("output.puml", "w") as f:
    f.write(plantuml_code)

In [16]:
from plantuml import PlantUML


In [17]:

!plantuml output.puml
